In [1]:
1+1

2

In [2]:
import os
%pwd

'd:\\Projects\\TextSummarizer\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Projects\\TextSummarizer'

## Basic configuration

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngesttionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories

## Configuration Updates

In [17]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self):
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngesttionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [18]:
import os
import urllib.request as request
import zipfile
from src.textsummarizer.logging import logger

## Components

In [19]:
class DataIngestion:
    def __init__(self, config:DataIngesttionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Data downloaded at {filename}")
        else:
            logger.info(f"file already exists at {self.config.local_data_file}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [20]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-02-09 17:35:08,829: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-09 17:35:08,832: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-09 17:35:08,834: INFO: common: created directory at: artifacts]
[2025-02-09 17:35:08,834: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-09 17:35:12,417: INFO: 1802843334: Data downloaded at artifacts/data_ingestion/data.zip]
